In [1]:
## Script to calculate summary statistics on all single-cell datasets and save them for usage in other scripts
# e.g. mean expression, percentage of zero of genes etc.

# Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/scgrn_R_4_1//lib/R/library"



Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: limma

Loading required package: mgcv

Loading required package: nlme

This is mgcv 1.9-1. For overview type 'help("mgcv-package")'.

Loading required package: genefilter

Loading required package: BiocParallel

corrplot 0.95 loaded

Loading required package: viridisLite


Attaching package: ‘purrr’


The following object is masked from ‘package:data.table’:

    transpose



Attaching package: ‘MASS’


The following object is masked from ‘package:genefilter’:

    area


The following object is masked from ‘package:biomaRt’:

    select



Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:data.table’:

    between, first, last


The following object is masked from ‘package:biomaRt’:

    selec

# Parameters

In [6]:
### Path to single-cell datasets (oneK1K, Wijst, UMCG2, UMCG3, multiome; stored as .Rds data files)

data_path_sc = "../data/current/wg3/"

In [8]:
### Path to save calculated summary statistics to

output_path_sc = "../data/current/summary_stats/"

# Functions

In [2]:
source('MS1_Functions.r')

# Data

## Load different sc dataset and get some meta-information

In [3]:
### Select datasets for which summary statistics should be calculated

In [11]:
folders = c(#'wg3_wijst2018'#, 
            #'wg3_Trynka', 
            'wg3_oneK1k'#,
           # 'wg3_NC2020',
            #'wg3_Nawijn',
            #'wg3_Li',
           # 'wg3_Ye',
           #  'wg3_okada',
           #   'wg3_sawcer',
          #  'wg3_Franke_split_v3'#,
          # 'wg3_Franke_split_v2'#,
          #  'wg3_blokland2020'
           )  ## specify folders that contain datasets that should be included

In [12]:
data = list()

In [13]:
overview_data = data.frame()

In [14]:
## Get all available datasets overview
for(i in folders){

    dataset_overview = data.frame( datasets = list.files(paste0(data_path_sc, i, '/input/L1')))
    if(nrow(dataset_overview) >= 1){
        dataset_overview$name = i
        dataset_overview = dataset_overview[!is.na(str_extract(dataset_overview$datasets, '.Rds')),]
        overview_data = rbind(overview_data, dataset_overview)
      }
    }


In [15]:
dataset_overview

,datasets,name
,<chr>,<chr>
1,B.Qced.Normalized.SCs.Rds,wg3_oneK1k
2,CD4_T.Qced.Normalized.SCs.Rds,wg3_oneK1k
3,CD8_T.Qced.Normalized.SCs.Rds,wg3_oneK1k
4,Mono.Qced.Normalized.SCs.Rds,wg3_oneK1k
5,NK.Qced.Normalized.SCs.Rds,wg3_oneK1k


In [16]:
head(overview_data,20)

,datasets,name
,<chr>,<chr>
1,B.Qced.Normalized.SCs.Rds,wg3_oneK1k
2,CD4_T.Qced.Normalized.SCs.Rds,wg3_oneK1k
3,CD8_T.Qced.Normalized.SCs.Rds,wg3_oneK1k
4,Mono.Qced.Normalized.SCs.Rds,wg3_oneK1k
5,NK.Qced.Normalized.SCs.Rds,wg3_oneK1k


In [17]:
overview_data$datasets_match = paste0(overview_data$name, overview_data$datasets)

In [18]:
overview_data[1,]

,datasets,name,datasets_match
,<chr>,<chr>,<chr>
1,B.Qced.Normalized.SCs.Rds,wg3_oneK1k,wg3_oneK1kB.Qced.Normalized.SCs.Rds


In [19]:
length(unique(overview_data$name))

[1] 1

In [20]:
nrow(overview_data)

[1] 5

In [21]:
## Function to load dataset
get_dataset = function(path){
    data = readRDS(path)
    }

In [22]:
## Get already generated summaries

In [23]:
available_summaries = data.frame()

In [24]:
available_summaries= data.frame( datasets = list.files(paste0(output_path_sc)))
available_summaries = data.frame(datasets = available_summaries[!is.na(str_extract(available_summaries$datasets, 'D1_Sample_cell_statistic')),])
    

In [25]:
available_summaries$datasets = str_replace(available_summaries$datasets, 'D1_Sample_cell_statistic', '')
available_summaries$datasets = str_replace(available_summaries$datasets, '.csv', '')

In [26]:
nrow(available_summaries)

[1] 174

In [27]:
head(available_summaries,2)

,datasets
,<chr>
1,wg3_blokland2020B.Qced.Normalized.SCs.Rds
2,wg3_blokland2020CD4_T.Qced.Normalized.SCs.Rds


In [28]:
nrow(overview_data[overview_data$datasets_match %in% available_summaries$datasets,]) # already existing summaries

[1] 4

In [29]:
## Remove already existing summaries to not recalculate them

In [30]:
calculate_new = TRUE

In [31]:
if(calculate_new == FALSE){
    overview_data = overview_data[!overview_data$datasets_match %in% available_summaries$datasets,]
    }
    

In [32]:
nrow(overview_data)

[1] 5

In [33]:
##Remove tmp Filtered objects

In [34]:
overview_data = overview_data[is.na(str_extract(overview_data$datasets, 'tmpFiltered')),]

In [35]:
#overview_data

In [36]:
head(overview_data,2)

,datasets,name,datasets_match
,<chr>,<chr>,<chr>
1,B.Qced.Normalized.SCs.Rds,wg3_oneK1k,wg3_oneK1kB.Qced.Normalized.SCs.Rds
2,CD4_T.Qced.Normalized.SCs.Rds,wg3_oneK1k,wg3_oneK1kCD4_T.Qced.Normalized.SCs.Rds


In [42]:
overview_data = overview_data[overview_data$datasets != 'B.Qced.Normalized.SCs.Rds',]

In [43]:
overview_data

,datasets,name,datasets_match
,<chr>,<chr>,<chr>
3,CD8_T.Qced.Normalized.SCs.Rds,wg3_oneK1k,wg3_oneK1kCD8_T.Qced.Normalized.SCs.Rds
5,NK.Qced.Normalized.SCs.Rds,wg3_oneK1k,wg3_oneK1kNK.Qced.Normalized.SCs.Rds


# Get summary statistics of input data and save

In [44]:
folders

[1] "wg3_oneK1k"

In [45]:
overview_data$data = overview_data$datasets

In [46]:
sort(unique(overview_data$data))

[1] "CD8_T.Qced.Normalized.SCs.Rds" "NK.Qced.Normalized.SCs.Rds"

In [47]:
choose_assay = 'data' # alternative: assay = 'data' or 'RNA' --> returns transformed values; whether to use normalized or raw counts

In [ ]:
### Generate empty dataframes to store calculated summaries in

In [48]:
sample_summary = data.frame()

In [49]:
sample_gene_statistic = data.frame()

In [50]:
sample_cell_statistic = data.frame()

In [51]:
overview_data$data[1]

[1] "CD8_T.Qced.Normalized.SCs.Rds"

In [52]:
overview_data

,datasets,name,datasets_match,data
,<chr>,<chr>,<chr>,<chr>
3,CD8_T.Qced.Normalized.SCs.Rds,wg3_oneK1k,wg3_oneK1kCD8_T.Qced.Normalized.SCs.Rds,CD8_T.Qced.Normalized.SCs.Rds
5,NK.Qced.Normalized.SCs.Rds,wg3_oneK1k,wg3_oneK1kNK.Qced.Normalized.SCs.Rds,NK.Qced.Normalized.SCs.Rds


In [ ]:
#### Calculate summary statistics

In [ ]:
for(i in 1:nrow(overview_data)){
    path = paste0(data_path_sc, overview_data$name[i], '/input/L1/', overview_data$data[i])
    data = get_dataset(path)   
    #print(data)
    #print(unique(data$Assignment))
    
    ## Get overview on amount samples: sample_summary
    sample_data = data[[]] %>% group_by(Assignment) %>% summarise(amount_cells = n())
    sample_data$dataset = overview_data$name[i]
    sample_data$cell_type = overview_data$data[i]
    sample_summary = rbind(sample_summary, sample_data)  # TBD outside of loop!


    for(k in unique(data$Assignment)) {     
            
        ## Get summary for each gene and sample: sample_gene_statistic
        sample_data = data[,data$Assignment == k ]
        #sample_data_trans = GetAssayData(sample_data ,assay = 'data')
        filter_data = GetAssayData(sample_data ,assay = choose_assay)   # need raw counts to determine at least 10 non-zero counts
        sample_data = GetAssayData(sample_data ,assay =choose_assay)  # alternative: assay = 'data' --> returns transformed values
     
            ### Filter genes with at leat 10 non-zero counts
            filter_data = filter_data[rowSums(filter_data !=  0) >= 10,]
            sample_data = sample_data[rownames(sample_data) %in% rownames(filter_data),]

            ### convert format
            sample_data = as.matrix(sample_data)
            #sample_data_trans = as.matrix(sample_data_trans)

            data_long = melt(sample_data)
            #data_long_trans = melt(sample_data_trans)

            colnames(data_long) = c('gene', 'cell', 'counts')
            #colnames(data_long_trans) = c('gene', 'cell', 'trans_counts')

            ### get summaries
            summary_data = data_long %>% group_by(gene) %>% summarise(
                amount_cells = n(),
                amount_zero_counts = sum(counts == 0),
                mean_counts = mean(counts, na.rm = TRUE),
                max_counts = max(counts, na.rm = TRUE),
                min_counts = min(counts, na.rm = TRUE),
                variance_counts = var(counts, na.rm = TRUE),
                #bin_size = as.numeric(try(fitdistr(counts, 'negative binomial')$estimate[1])), 
                #bin_mu= as.numeric(try( fitdistr(counts, 'negative binomial')$estimate[2]))

            )

            summary_data$percentage_zero = summary_data$amount_zero_counts / summary_data$amount_cells
            summary_data$amount_non_zero_counts = summary_data$amount_cells - summary_data$amount_zero_counts
            summary_data$percentage_zero =  summary_data$percentage_zero*100
            ## bin different columns
            summary_data =  summary_data %>% mutate(amount_cells_bin = cut(amount_cells, breaks=c(0,25, 50, 100,250, 500, 750, 1500)))
            summary_data =  summary_data %>% mutate(percentage_zero_bin = cut(percentage_zero, breaks=c(-0.001,5, 10, 20,40, 60, 80,90, 95, 100)))


            summary_data$sample = k
            summary_data$cell_type = overview_data$data[i]
            summary_data$dataset = overview_data$name[i]

            sample_gene_statistic = rbind(sample_gene_statistic, summary_data)
        #
        
        # Get summaries for each sample and each cell: sample_cell_statistic
        sample_data = data[,data$Assignment == k ]
        sample_data = GetAssayData(sample_data ,assay =choose_assay)  # alternative: assay = 'data' --> returns transformed values

            ### convert format
            sample_data = as.matrix(sample_data)

            data_long = melt(sample_data)

            colnames(data_long) = c('gene', 'cell', 'counts')

            ### get summaries
            summary_data = data_long %>% group_by(cell) %>% summarise(
                amount_counts = sum(counts),
                amount_genes = n(),
                amount_zero_counts = sum(counts == 0),
                mean_counts = mean(counts,na.rm = TRUE),
                max_counts = max(counts,na.rm = TRUE),
                min_counts = min(counts,na.rm = TRUE),
                variance_counts = var(counts,na.rm = TRUE))

            summary_data$percentage_zero = summary_data$amount_zero_counts / summary_data$amount_genes
            summary_data$amount_non_zero_counts = summary_data$amount_genes - summary_data$amount_zero_counts


            summary_data$sample = k
            summary_data$cell_type = overview_data$data[i]
            summary_data$dataset = overview_data$name[i]

            sample_cell_statistic = rbind(sample_cell_statistic, summary_data)
        
            write.csv(sample_summary, paste0(output_path_sc, 'D1_Sample_summary', overview_data$name[i], overview_data$data[i], '_', choose_assay, '.csv'))
            write.csv(sample_gene_statistic, paste0(output_path_sc, 'D1_Sample_gene_statistic', overview_data$name[i], overview_data$data[i], '_', choose_assay, '.csv'))
            write.csv(sample_cell_statistic, paste0(output_path_sc, 'D1_Sample_cell_statistic', overview_data$name[i], overview_data$data[i], '_', choose_assay, '.csv'))
            
            flush.console()
            print(k)
            flush.console()
                
            
            
        
        
        }
        flush.console()
        print(i)
        flush.console()
    
    
        sample_summary = data.frame()
        sample_gene_statistic = data.frame()
        sample_cell_statistic = data.frame()
    }
        
    

[1] "685"
[1] "691"
[1] "689"
[1] "684"
[1] "688"
[1] "690"
[1] "694"
[1] "692"
[1] "686"
[1] "687"
[1] "693"
[1] "683"
[1] "832"
[1] "824"
[1] "831"
[1] "830"
[1] "833"
[1] "825"
[1] "844"
[1] "843"
[1] "845"
[1] "827"
[1] "829"
[1] "826"
[1] "828"
[1] "849"
[1] "835"
[1] "850"
[1] "848"
[1] "991"
[1] "851"
[1] "847"
[1] "990"
[1] "852"
[1] "986"
[1] "998"
[1] "837"
[1] "836"
[1] "846"
[1] "1000"
[1] "993"
[1] "813"
[1] "838"
[1] "996"
[1] "812"
[1] "994"
[1] "999"
[1] "997"
[1] "995"
[1] "992"
[1] "811"
[1] "853"
[1] "987"
[1] "856"
[1] "988"
[1] "858"
[1] "857"
[1] "1005"
[1] "1003"
[1] "855"
[1] "859"
[1] "1002"
[1] "1001"
[1] "1004"
[1] "989"
[1] "860"
[1] "854"
[1] "805"
[1] "1008"
[1] "862"
[1] "1007"
[1] "804"
[1] "864"
[1] "892"
[1] "863"
[1] "803"
[1] "861"
[1] "891"
[1] "865"
[1] "1006"
[1] "806"
[1] "1011"
[1] "893"
[1] "807"
[1] "1014"
[1] "1012"
[1] "1013"
[1] "809"
[1] "808"
[1] "1015"
[1] "1009"
[1] "810"
[1] "894"
[1] "1010"
[1] "1017"
[1] "1019"
[1] "1022"
[1] "1023"


In [ ]:
### Implementation for multiome data; needs to be run differently (multiple observations per subject)

In [ ]:
#### Path to multiome data
path = "../data/current/wg3/wg3_multiome/input/L1/CD4_T.Qced.Normalized.SCs.Rds"

In [35]:
name = 'wg3_multiome'

In [36]:
output_path_sc  =  "../data/current/summary_stats/"